# Study of the comfort parameter

The objective of this study is to determine the influence of the comfort parameter $\alpha$ on the power distribution. We will now focus on the dynamic model which is described relies on DMPC.py.

One of the principal issue here is to get an overview of the compfort parameter in ordr to use it to cheat in the DMPC.


-- Sylvain Chatel, July 2016 --

In [2]:
from __future__ import division, print_function
import DMPC_Draft as DO
from cvxopt import matrix, solvers
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
from tempfile import TemporaryFile
%matplotlib inline

/Users/sylvainchatel/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Influence of the comfort parameter for two users

### Definition of the problem

In [3]:
m = 2

# Time step
dt = 0.1  # h
# Simulation Horizon
N_sim = int(24/dt)
# Prediction Horizon
N = int(5/dt)
# max energy in kW
Umax = 3
# max admissible energy
u_m = np.array([2, 2], dtype=float)
# thermal parameters
        ## Exterior Temperature on the whole horizon
Text_sim = np.ones(m*(N_sim + N))*0
        ## Exterior Temperature on the prediction horizon initially
Text = Text_sim[0:m*N]
        ## Temperature when present
Tpres = 22
        ## Temperature when absent
Tabs = 18
        ## Initial temperature in all users
T_init = np.array([10, 10], dtype=float)
        ## Thermal Resistance
Rth = np.array([50, 50], dtype=float)
        ## Thermal Capacity
Cth = np.array([0.056, 0.056], dtype=float)
        ## Reference temperature through the whole horizon
T_mod = np.hstack(
    (DO.temp_id(N_sim + N, Tabs, Tpres), DO.temp_id(N_sim + N, Tabs, Tpres)))  ## ATTENTION : defined user after user
        ## Reference temperature through the simulation horizon
T_id_pred = np.hstack(
    (DO.temp_id(N_sim, Tabs, Tpres), DO.temp_id(N_sim, Tabs, Tpres)))  ## ATTENTION : defined user after user
# comfort factor
alpha_1 = np.array([10, 10], dtype=float)
alpha_2 = np.array([10, 20], dtype=float)

## Verifications
assert len(u_m) == m, "illegal number of users. Expecting %s. and received %s." % (m, len(u_m))
assert len(T_init) == m, "illegal number of T_init. Expecting %s. and received %s." % (m, len(T_init))
assert len(Rth) == m, "illegal number of Rth. Expecting %s. and received %s." % (m, len(Rth))
assert len(Cth) == m, "illegal number of Cth. Expecting %s. and received %s." % (m, len(Cth))
assert len(alpha) == m, "illegal number of alpha. Expecting %s. and received %s." % (m, len(alpha))

# Definition of the dictionary
pb_1 = dict(m=m, dt=dt, Umax=Umax, u_m=u_m, Text=Text, Text_sim=Text_sim, T_mod=T_mod, T_init=T_init, Rth=Rth, Cth=Cth,
          T_id_pred=T_id_pred, alpha=alpha, N=N, N_sim=N_sim)

pb_2 = dict(m=m, dt=dt, Umax=Umax, u_m=u_m, Text=Text, Text_sim=Text_sim, T_mod=T_mod, T_init=T_init, Rth=Rth, Cth=Cth,
          T_id_pred=T_id_pred, alpha=alpha_2, N=N, N_sim=N_sim)

NameError: global name 'dt' is not defined

In [ ]:
T_1, U_1 =get_Opt_CL(pb_1)
T_2, U_2= get_Opt_CL(pb_2)
plot_T(pb, 0, T_res, U, r'$\alpha$', T_cen, U_cen, r'$\alpha_2$')